<h1> QUESTION 1: Postal Code Data Frame from WIKI Page </h1>

In [124]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import lxml
import numpy as np

In [130]:
# I Struggled with beautfulsoup packageing in getting the rows logged into the DF,
# so i found a much simplier way using Pandas

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
table = pd.read_html(url)[0]
table.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [148]:
#Start of the 1st clean through removing 'not assinged'

table = table.astype('str')
table2 = table[table.Borough != "Not assigned"]
table2.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [157]:
#Check for Null values in Neighbourhood

table2['Neighborhood'].isnull().values.any()

False

!!! with no Null values in neighjbourhood its not neccesary to code out the nulls for the Borough name

In [160]:
table2.shape

(103, 3)